In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from models import CNN_Clasificator, LSTM__Clasificator
from get_embedded_data import get_data_glove_CNN, get_data_glove_LSTM, get_data_tokenizer_MLP, split_data, MAPPING

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Python311\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Python311\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warn

In [2]:
device = torch.device("cuda")
device

device(type='cuda')

In [3]:
# max_len_sentence = 0
# pos = 0
# for i, sentence in enumerate(list_of_words):
#     max_len_sentence, pos = (len(sentence), i) if len(sentence)>max_len_sentence else (max_len_sentence, pos)
# max_len_sentence, pos

In [4]:
# import nltk
# from nltk.tokenize import word_tokenize
# from gensim.models import Word2Vec
# # nltk.download('punkt')

# import gensim.downloader as api
# corpus = api.load('text8')

# with open('data_set.csv', 'r', encoding='utf-8') as dh:
#     list_of_words = []
#     list_of_targets = []
#     for i, line in enumerate(dh):
#         if i > 0:
#             line = line.strip()
#             line = line.split('@')
#             line[-1] = word_tokenize(line[-1].lower())
#             list_of_words.append(line[-1])
#             list_of_targets.append(line[1])
#     dh.close()

# emb_dim = 100

# model = Word2Vec(corpus, min_count=1)
# model.build_vocab(list_of_words, update=True)
# model.train(list_of_words, total_examples=model.corpus_count, epochs=20)
# model.save('word2vec_100d')

# # model = Word2Vec.load('word2vec/word2vec_100d')

# # Access embeddings
# word_embeddings = model.wv
# print(model.wv.most_similar("folly"))
# print(word_embeddings['superstitious'])


In [5]:
# word2vec_embeddings = []
# for sentence in list_of_words:
#     emb_sentence = np.empty((100,0))
#     for word in sentence:
#         emb_sentence = np.hstack((emb_sentence, np.reshape(model.wv[word], (100, 1))))
#     word2vec_embeddings.append(torch.from_numpy(emb_sentence))

In [6]:
# import torchtext
# import torchtext.vocab

# glove = torchtext.vocab.GloVe(name="6B", dim=100)

In [7]:
# list_of_inc = []
# for i, sen in enumerate(list_of_words):
#     counter = 0
#     for j, wor in enumerate(sen):
#         if wor not in glove:
#             counter += 1
#     if counter >= 1: list_of_inc.append((i, counter))
# print(len(list_of_inc))
# list_of_inc

In [8]:
# abc = []
# for i, sen in enumerate(list_of_words):
#     for j, wor in enumerate(sen):
#             if wor not in glove: abc.append((list_of_targets[i],i, j, wor))

In [9]:
# miss_by_aut=[]
# i = 0
# inc = 0
# prev_aut = list_of_inc[0][0]
# for aut , _, _ in list_of_inc:
#     if aut == prev_aut:
#         inc += 1
#     else:
#         miss_by_aut.append([prev_aut, inc])
#         prev_aut = aut
#         i+=1
#         inc = 0
# miss_by_aut.append([aut, inc])
# miss_by_aut

In [10]:
# import nltk

In [11]:
# nltk.download()

In [12]:
X_train, X_test, y_train, y_test = split_data("data_set.csv", "author", "quote", test_size=0.2, separator="@", mapping=MAPPING, labels_to_delete=[])

['aristotle' 'freud' 'hegel' 'kant' 'nietzsche' 'plato' 'sartre'
 'schopenhauer' 'spinoza']


In [13]:
# dataloader = get_data_BERT_MLP(20, device)
train_dataloader = get_data_glove_CNN(200, X_train, y_train)
test_dataloader = get_data_glove_CNN(200, X_test, y_test)

In [14]:
network = CNN_Clasificator().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters())

In [15]:
max_epoch = 30
network.train()
for epoch in range(max_epoch):

    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = network(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        total += labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    print('[%d/%d] loss: %.3f accuracy: %d' %
          (epoch+1, max_epoch, running_loss / 2000, 100 * correct / total))
    running_loss = 0.0

print('Finished Training')

correct = 0
total = 0
for i, data in enumerate(test_dataloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = network(inputs)
    total += labels.size(0)
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == labels).sum().item()

print(f"Test acc: {100 * correct / total}")

c:\Python311\Lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


[1/30] loss: 0.010 accuracy: 17
[2/30] loss: 0.009 accuracy: 21
[3/30] loss: 0.009 accuracy: 22
[4/30] loss: 0.009 accuracy: 22
[5/30] loss: 0.009 accuracy: 23
[6/30] loss: 0.009 accuracy: 26
[7/30] loss: 0.009 accuracy: 28
[8/30] loss: 0.008 accuracy: 31
[9/30] loss: 0.008 accuracy: 34
[10/30] loss: 0.008 accuracy: 38
[11/30] loss: 0.008 accuracy: 39
[12/30] loss: 0.007 accuracy: 43
[13/30] loss: 0.007 accuracy: 45
[14/30] loss: 0.007 accuracy: 46
[15/30] loss: 0.007 accuracy: 49
[16/30] loss: 0.006 accuracy: 55
[17/30] loss: 0.006 accuracy: 56
[18/30] loss: 0.006 accuracy: 60
[19/30] loss: 0.005 accuracy: 62
[20/30] loss: 0.005 accuracy: 66
[21/30] loss: 0.005 accuracy: 67
[22/30] loss: 0.004 accuracy: 70
[23/30] loss: 0.004 accuracy: 71
[24/30] loss: 0.004 accuracy: 76
[25/30] loss: 0.004 accuracy: 77
[26/30] loss: 0.003 accuracy: 78
[27/30] loss: 0.003 accuracy: 80
[28/30] loss: 0.003 accuracy: 83
[29/30] loss: 0.003 accuracy: 84
[30/30] loss: 0.002 accuracy: 87
Finished Training
T

In [16]:
train_dataloader = get_data_glove_LSTM(200, X_train, y_train)
test_dataloader = get_data_glove_LSTM(1, X_test, y_test)

In [17]:
network = LSTM__Clasificator(80).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters())

In [18]:
max_epoch = 70
network.train()
for epoch in range(max_epoch):

    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels, x_len = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        network.prep(200)
        outputs = network(inputs)

        new_preds = torch.empty((0,9)).to(device)
        for i in range(len(outputs)):
            new_preds = torch.vstack((new_preds, outputs[i][x_len[i]-1]))

        loss = criterion(new_preds, labels)
        loss.backward()
        optimizer.step()


        total += labels.size(0)
        _, predicted = torch.max(new_preds.data, 1)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    val_correct = 0
    val_total = 0
    for i, data in enumerate(test_dataloader, 0):
        inputs, labels, x_len = data
        inputs, labels = inputs.to(device), labels.to(device)
        network.prep(1)
        outputs = network(inputs)
        new_preds = torch.empty((0,9)).to(device)
        for i in range(len(outputs)):
            new_preds = torch.vstack((new_preds, outputs[i][x_len[i]-1]))
        val_total += labels.size(0)
        _, predicted = torch.max(new_preds.data, 1)
        val_correct += (predicted == labels).sum().item()
    print(f"Test accuracy: {100 * val_correct / val_total}")

    print('[%d/%d] loss: %.3f accuracy: %d' %
          (epoch+1, max_epoch, running_loss / 2000, 100 * correct / total))
    running_loss = 0.0

c:\Users\bgale\Desktop\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens


Test accuracy: 16.80161943319838
[1/70] loss: 0.010 accuracy: 16
Test accuracy: 18.42105263157895
[2/70] loss: 0.009 accuracy: 18
Test accuracy: 20.040485829959515
[3/70] loss: 0.009 accuracy: 18
Test accuracy: 19.4331983805668
[4/70] loss: 0.009 accuracy: 19
Test accuracy: 21.05263157894737
[5/70] loss: 0.009 accuracy: 22
Test accuracy: 25.303643724696357
[6/70] loss: 0.009 accuracy: 21
Test accuracy: 25.910931174089068
[7/70] loss: 0.009 accuracy: 25
Test accuracy: 28.94736842105263
[8/70] loss: 0.009 accuracy: 26
Test accuracy: 24.089068825910932
[9/70] loss: 0.009 accuracy: 26
Test accuracy: 24.898785425101213
[10/70] loss: 0.009 accuracy: 28
Test accuracy: 24.089068825910932
[11/70] loss: 0.009 accuracy: 27
Test accuracy: 27.530364372469634
[12/70] loss: 0.009 accuracy: 26
Test accuracy: 27.93522267206478
[13/70] loss: 0.009 accuracy: 29
Test accuracy: 28.13765182186235
[14/70] loss: 0.008 accuracy: 31
Test accuracy: 28.94736842105263
[15/70] loss: 0.008 accuracy: 32
Test accuracy

In [19]:
correct = 0
total = 0
for i, data in enumerate(test_dataloader, 0):
    inputs, labels, x_len = data
    inputs, labels = inputs.to(device), labels.to(device)
    network.prep(1)
    outputs = network(inputs)
    new_preds = torch.empty((0,9)).to(device)
    for i in range(len(outputs)):
        new_preds = torch.vstack((new_preds, outputs[i][x_len[i]-1]))
    total += labels.size(0)
    _, predicted = torch.max(new_preds.data, 1)
    correct += (predicted == labels).sum().item()

print(f"Test acc: {100 * correct / total}")

Test acc: 32.18623481781376
